<a href="https://colab.research.google.com/github/ykitaguchi77/YOLACT_Mobius/blob/main/YOLOv8_training_LR_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#**Mobius YOLO8 training (左右判定あり)**

```
# Classes
names:
  0: eyelid_R
  1: iris_R
  2: pupil_R
  3: eyelid_L
  4: iris_L
  5: pupil_L
```

In [2]:
#@title #ライブラリのインストール
#@markdown [shift]+[Enter]を押して起動してください
from google.colab import drive
drive.mount('/content/drive')
import os
import cv2
import numpy as np
import matplotlib.pyplot as plt
from skimage import measure
import json
from tqdm import tqdm  # tqdmをインポート
import os
import matplotlib

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
#@title #ディレクトリと座標の個数
#@markdown 指定を終えてから、[shift]+[Enter]を押して起動してください。
#@markdown <br> 学習に適切な個数が分からなかったので、変更できるようにしました。

# 入力ディレクトリと出力ディレクトリを指定してください
input_dir = "/content/drive/MyDrive/Deep_learning/Eyelid_segmentation/MOBIUS/Masks/1"
output_dir = "/content"

# eyelid, iris, pupilごとに座標の個数を指定してください。
num_points_eyelid = 24
num_points_iris = 16
num_points_pupil = 12

In [ ]:
#@title #Mobius to label起動
#@markdown 　[shift]+[Enter]を押して起動してください。
#@markdown　<br>うまくいっていないラベルも存在するので、目視で確認して除外すべきかもしれません。
def create_mask(image, color):
    return np.all(image == color, axis=-1)

def apply_mask(image, mask):
    return np.where(mask, 255, 0)

def extract_contours(img, _num_points):
    contours = measure.find_contours(img, 0.5)
    points_list = []

    for contour in contours:
        contour_length = len(contour)

        if contour_length > _num_points:
            step = contour_length // _num_points
            selected_points = [contour[i * step] for i in range(_num_points)]
        else:
            selected_points = contour

        points = [[float(point[1]), float(point[0])] for point in selected_points]
        points_list.append(points)

    return points_list

def create_custom_output(points_list_blue, points_list_blue_green, points_list_all_colors, img_width, img_height, output_file):
    with open(output_file, 'w') as f:
        # 0: eyelid
        f.write("0 ")
        for point in points_list_all_colors[0]:
            x = point[0] / img_width
            y = point[1] / img_height
            f.write(f"{x} {y} ")
        f.write("\n")

        # 1: iris
        f.write("1 ")
        for point in points_list_blue_green[0]:
            x = point[0] / img_width
            y = point[1] / img_height
            f.write(f"{x} {y} ")
        f.write("\n")

        # 2: pupil
        f.write("2 ")
        for point in points_list_blue[0]:
            x = point[0] / img_width
            y = point[1] / img_height
            f.write(f"{x} {y} ")
        f.write("\n")

def process_image(mask_file, output_dir, num_points_eyelid, num_points_iris, num_points_pupil):
    img = cv2.imread(mask_file, cv2.IMREAD_COLOR)
    img_height, img_width = img.shape[:2]

    mask_blue = create_mask(img, [255, 0, 0])
    mask_green = create_mask(img, [0, 255, 0])
    mask_red = create_mask(img, [0, 0, 255])

    mask_blue_green = np.logical_or(mask_blue, mask_green)
    mask_all_colors = np.logical_or(mask_blue_green, mask_red)

    img_blue = apply_mask(img, mask_blue)
    img_blue_green = apply_mask(img, mask_blue_green)
    img_all_colors = apply_mask(img, mask_all_colors)

    # 各ラベルごとに異なる個数の座標を抽出
    points_list_blue = extract_contours(img_blue, num_points_pupil)
    points_list_blue_green = extract_contours(img_blue_green, num_points_iris)
    points_list_all_colors = extract_contours(img_all_colors, num_points_eyelid)

    # 出力先ディレクトリとファイル名を設定
    base_name = os.path.splitext(os.path.basename(mask_file))[0]
    output_file = os.path.join(output_dir, base_name + ".txt")

    # カスタム出力
    create_custom_output(points_list_blue, points_list_blue_green, points_list_all_colors, img_width, img_height, output_file)

    # 書き換え部分
    x_eyelid = [point[0] for point in points_list_all_colors[0]]
    y_eyelid = [point[1] for point in points_list_all_colors[0]]

    x_iris = [point[0] for point in points_list_blue_green[0]]
    y_iris = [point[1] for point in points_list_blue_green[0]]

    x_pupil = [point[0] for point in points_list_blue[0]]
    y_pupil = [point[1] for point in points_list_blue[0]]

    # 元の画像を読み込む
    img_path = mask_file.replace("Masks", "Images").replace(".png", ".jpg")
    img = cv2.imread(img_path, cv2.IMREAD_COLOR)

    # プロット作成
    plt.figure(figsize=(12, 6))

    # 左側に元の画像を表示
    plt.subplot(1, 2, 1)
    plt.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
    plt.title(base_name)  # ファイル名をタイトルに設定

    # 右側に散布図を表示
    plt.subplot(1, 2, 2)
    plt.scatter(x_eyelid, y_eyelid, label='eyelid', color='blue')
    plt.scatter(x_iris, y_iris, label='iris', color='green')
    plt.scatter(x_pupil, y_pupil, label='pupil', color='red')
    plt.title("Contours")  # 輪郭

    # 軸を画像の大きさに合わせて調整
    plt.xlim([0, img_width])
    plt.ylim([img_height, 0])  # Y軸の反転

    # 縦横の比率を保つ
    plt.gca().set_aspect('equal', adjustable='box')

    # 凡例を表示する
    plt.legend()

    # レイアウトを調整して表示
    plt.tight_layout()
    plt.show()





def process_all_images_in_directory(input_dir, output_dir, num_points_eyelid, num_points_iris, num_points_pupil):
    # 入力ディレクトリ内のすべての.pngファイルを処理
    png_files = [file_name for file_name in os.listdir(input_dir) if file_name.endswith(".png")]

    # tqdmを使って進行状況を表示
    for file_name in tqdm(png_files, desc="Processing images"):
        mask_file = os.path.join(input_dir, file_name)
        process_image(mask_file, output_dir, num_points_eyelid, num_points_iris, num_points_pupil)



if __name__ == '__main__':
    process_all_images_in_directory(input_dir, output_dir, num_points_eyelid, num_points_iris, num_points_pupil)


In [ ]:
#@title #**Mobius mask to label (LR version)**
import os
import cv2
import numpy as np
from skimage import measure
import matplotlib.pyplot as plt
from tqdm import tqdm

def create_mask(image, color):
    return np.all(image == color, axis=-1)

def apply_mask(image, mask):
    return np.where(mask, 255, 0)

def extract_contours(img, _num_points):
    contours = measure.find_contours(img, 0.5)
    points_list = []

    for contour in contours:
        contour_length = len(contour)

        if contour_length > _num_points:
            step = contour_length // _num_points
            selected_points = [contour[i * step] for i in range(_num_points)]
        else:
            selected_points = contour

        points = [[float(point[1]), float(point[0])] for point in selected_points]
        points_list.append(points)

    return points_list

def get_eye_side(filename):
    basename = os.path.basename(filename)
    name_without_extension = os.path.splitext(basename)[0]
    parts = name_without_extension.split('_')

    if len(parts) >= 3:
        relevant_part = parts[2]
        first_char = relevant_part[0] if relevant_part else None

        if first_char in ['R', 'L']:
            return first_char

    return None

def create_custom_output(points_list_blue, points_list_blue_green, points_list_all_colors, img_width, img_height, output_file, eye_side):
    with open(output_file, 'w') as f:
        # eyelid
        label = "0" if eye_side == 'R' else "3"
        f.write(f"{label} ")
        for point in points_list_all_colors[0]:
            x = point[0] / img_width
            y = point[1] / img_height
            f.write(f"{x} {y} ")
        f.write("\n")

        # iris
        label = "1" if eye_side == 'R' else "4"
        f.write(f"{label} ")
        for point in points_list_blue_green[0]:
            x = point[0] / img_width
            y = point[1] / img_height
            f.write(f"{x} {y} ")
        f.write("\n")

        # pupil
        label = "2" if eye_side == 'R' else "5"
        f.write(f"{label} ")
        for point in points_list_blue[0]:
            x = point[0] / img_width
            y = point[1] / img_height
            f.write(f"{x} {y} ")
        f.write("\n")

def process_image(mask_file, output_dir, num_points_eyelid, num_points_iris, num_points_pupil):
    img = cv2.imread(mask_file, cv2.IMREAD_COLOR)
    img_height, img_width = img.shape[:2]

    mask_blue = create_mask(img, [255, 0, 0])
    mask_green = create_mask(img, [0, 255, 0])
    mask_red = create_mask(img, [0, 0, 255])

    mask_blue_green = np.logical_or(mask_blue, mask_green)
    mask_all_colors = np.logical_or(mask_blue_green, mask_red)

    img_blue = apply_mask(img, mask_blue)
    img_blue_green = apply_mask(img, mask_blue_green)
    img_all_colors = apply_mask(img, mask_all_colors)

    points_list_blue = extract_contours(img_blue, num_points_pupil)
    points_list_blue_green = extract_contours(img_blue_green, num_points_iris)
    points_list_all_colors = extract_contours(img_all_colors, num_points_eyelid)

    base_name = os.path.splitext(os.path.basename(mask_file))[0]
    output_file = os.path.join(output_dir, base_name + ".txt")

    eye_side = get_eye_side(mask_file)
    if eye_side is None:
        print(f"Warning: Could not determine eye side for {mask_file}")
        eye_side = 'R'  # Default to right eye if unable to determine

    create_custom_output(points_list_blue, points_list_blue_green, points_list_all_colors, img_width, img_height, output_file, eye_side)

    # Visualization
    x_eyelid = [point[0] for point in points_list_all_colors[0]]
    y_eyelid = [point[1] for point in points_list_all_colors[0]]
    x_iris = [point[0] for point in points_list_blue_green[0]]
    y_iris = [point[1] for point in points_list_blue_green[0]]
    x_pupil = [point[0] for point in points_list_blue[0]]
    y_pupil = [point[1] for point in points_list_blue[0]]

    img_path = mask_file.replace("Masks", "Images").replace(".png", ".jpg")
    img = cv2.imread(img_path, cv2.IMREAD_COLOR)

    plt.figure(figsize=(12, 6))
    plt.subplot(1, 2, 1)
    plt.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
    plt.title(base_name)

    plt.subplot(1, 2, 2)
    plt.scatter(x_eyelid, y_eyelid, label='eyelid', color='blue')
    plt.scatter(x_iris, y_iris, label='iris', color='green')
    plt.scatter(x_pupil, y_pupil, label='pupil', color='red')
    plt.title("Contours")

    plt.xlim([0, img_width])
    plt.ylim([img_height, 0])
    plt.gca().set_aspect('equal', adjustable='box')
    plt.legend()
    plt.tight_layout()
    plt.show()

def process_all_images_in_directory(input_dir, output_dir, num_points_eyelid, num_points_iris, num_points_pupil):
    os.makedirs(output_dir, exist_ok=True)
    png_files = [file_name for file_name in os.listdir(input_dir) if file_name.endswith(".png")]

    for file_name in tqdm(png_files, desc="Processing images"):
        mask_file = os.path.join(input_dir, file_name)
        process_image(mask_file, output_dir, num_points_eyelid, num_points_iris, num_points_pupil)

if __name__ == '__main__':
    # Configuration
    input_dir = "/content/drive/MyDrive/Deep_learning/Eyelid_segmentation/MOBIUS/Masks/1"
    output_dir = "/content"

    num_points_eyelid = 24
    num_points_iris = 16
    num_points_pupil = 12

    # Process all images
    process_all_images_in_directory(input_dir, output_dir, num_points_eyelid, num_points_iris, num_points_pupil)

    print(f"Processing complete. Output files are saved in: {output_dir}")

#**YOLOv8 instance segmentation training**

In [ ]:
"""
project/
│
├── data/
│   ├── train/
│   │   ├── images/
│   │   └── labels/
│   │
│   ├── val/
│   │   ├── images/
│   │   └── labels/
│   │
│   └── test/  (オプション)
│       ├── images/
│       └── labels/
│
│
├── dataset.yaml
│
├── models/
│
└── runs/  (トレーニング結果が保存される)
"""

###**Create dataset**

In [8]:
"""
このスクリプトは、Mobiusデータセットの画像とマスクをYOLOv8形式に変換します。

主な機能：
1. Masksディレクトリ内の全てのマスク画像を処理（フォルダ番号順）
2. 各マスクに対応するYOLOv8形式のラベルファイルを生成
3. 対応する前眼部画像を指定された出力ディレクトリにコピー
4. 処理中に画像とマスクのプロットを表示
5. tqdmを使用して処理の進捗を表示
6. 開始フォルダを指定可能

入力ディレクトリ構造：
- Masks: /content/drive/MyDrive/Deep_learning/Eyelid_segmentation/MOBIUS/Masks/
  - 1/
  - 2/
  ...
  - 35/
- Images: /content/drive/MyDrive/Deep_learning/Eyelid_segmentation/MOBIUS/Images/
  - 1/
  - 2/
  ...
  - 35/

出力ディレクトリ構造：
- Labels: /content/drive/MyDrive/Deep_learning/Eyelid_segmentation/MOBIUS/YOLOv8_training_LR/labels/
- Images: /content/drive/MyDrive/Deep_learning/Eyelid_segmentation/MOBIUS/YOLOv8_training_LR/images/

注意：
- 出力ディレクトリが存在しない場合は自動的に作成されます
- マスクの色に基づいて、瞼（赤）、虹彩（緑）、瞳孔（青）を識別します
- 各部位の座標点の数は変数で設定可能です（デフォルト：瞼24点、虹彩16点、瞳孔12点）
- フォルダは番号順に処理されます
- 開始フォルダ番号を指定できます
- 左右の目を識別し、それぞれに異なるラベルを割り当てます（右目：0,1,2、左目：3,4,5）
"""


import os
import cv2
import numpy as np
from skimage import measure
import matplotlib.pyplot as plt
from tqdm import tqdm
import shutil

def create_mask(image, color):
    return np.all(image == color, axis=-1)

def apply_mask(image, mask):
    return np.where(mask, 255, 0)

def extract_contours(img, _num_points):
    contours = measure.find_contours(img, 0.5)
    points_list = []

    for contour in contours:
        contour_length = len(contour)

        if contour_length > _num_points:
            step = contour_length // _num_points
            selected_points = [contour[i * step] for i in range(_num_points)]
        else:
            selected_points = contour

        points = [[float(point[1]), float(point[0])] for point in selected_points]
        points_list.append(points)

    return points_list

def get_eye_side(filename):
    basename = os.path.basename(filename)
    name_without_extension = os.path.splitext(basename)[0]
    parts = name_without_extension.split('_')

    if len(parts) >= 3:
        relevant_part = parts[2]
        first_char = relevant_part[0] if relevant_part else None

        if first_char in ['R', 'L']:
            return first_char

    return None

def create_custom_output(points_list_blue, points_list_blue_green, points_list_all_colors, img_width, img_height, output_file, eye_side):
    with open(output_file, 'w') as f:
        # eyelid
        label = "0" if eye_side == 'R' else "3"
        f.write(f"{label} ")
        for point in points_list_all_colors[0]:
            x = point[0] / img_width
            y = point[1] / img_height
            f.write(f"{x:.6f} {y:.6f} ")
        f.write("\n")

        # iris
        label = "1" if eye_side == 'R' else "4"
        f.write(f"{label} ")
        for point in points_list_blue_green[0]:
            x = point[0] / img_width
            y = point[1] / img_height
            f.write(f"{x:.6f} {y:.6f} ")
        f.write("\n")

        # pupil
        label = "2" if eye_side == 'R' else "5"
        f.write(f"{label} ")
        for point in points_list_blue[0]:
            x = point[0] / img_width
            y = point[1] / img_height
            f.write(f"{x:.6f} {y:.6f} ")
        f.write("\n")

def process_image(mask_file, images_dir, output_labels_dir, output_images_dir, num_points_eyelid, num_points_iris, num_points_pupil):
    img = cv2.imread(mask_file, cv2.IMREAD_COLOR)
    img_height, img_width = img.shape[:2]

    mask_blue = create_mask(img, [255, 0, 0])
    mask_green = create_mask(img, [0, 255, 0])
    mask_red = create_mask(img, [0, 0, 255])

    mask_blue_green = np.logical_or(mask_blue, mask_green)
    mask_all_colors = np.logical_or(mask_blue_green, mask_red)

    img_blue = apply_mask(img, mask_blue)
    img_blue_green = apply_mask(img, mask_blue_green)
    img_all_colors = apply_mask(img, mask_all_colors)

    points_list_blue = extract_contours(img_blue, num_points_pupil)
    points_list_blue_green = extract_contours(img_blue_green, num_points_iris)
    points_list_all_colors = extract_contours(img_all_colors, num_points_eyelid)

    base_name = os.path.splitext(os.path.basename(mask_file))[0]
    output_file = os.path.join(output_labels_dir, base_name + ".txt")

    eye_side = get_eye_side(mask_file)
    if eye_side is None:
        print(f"Warning: Could not determine eye side for {mask_file}")
        eye_side = 'R'  # Default to right eye if unable to determine

    create_custom_output(points_list_blue, points_list_blue_green, points_list_all_colors, img_width, img_height, output_file, eye_side)

    # Copy corresponding image
    folder_number = os.path.basename(os.path.dirname(mask_file))
    image_path = os.path.join(images_dir, folder_number, base_name + ".jpg")
    if os.path.exists(image_path):
        shutil.copy(image_path, output_images_dir)
    else:
        print(f"Warning: Corresponding image not found for {base_name}")

def process_all_images_in_directory(masks_dir, images_dir, output_labels_dir, output_images_dir, num_points_eyelid, num_points_iris, num_points_pupil, start_folder):
    def folder_num(name):
        try:
            return int(name)
        except ValueError:
            return float('inf')

    folders = sorted([f for f in os.listdir(masks_dir) if os.path.isdir(os.path.join(masks_dir, f))], key=folder_num)

    start_index = folders.index(str(start_folder)) if str(start_folder) in folders else 0
    folders = folders[start_index:]

    for folder_name in folders:
        folder_path = os.path.join(masks_dir, folder_name)
        png_files = [file_name for file_name in os.listdir(folder_path) if file_name.endswith(".png")]
        for file_name in tqdm(png_files, desc=f"Processing folder {folder_name}"):
            mask_file = os.path.join(folder_path, file_name)
            process_image(mask_file, images_dir, output_labels_dir, output_images_dir, num_points_eyelid, num_points_iris, num_points_pupil)

if __name__ == '__main__':
    masks_dir = "/content/drive/MyDrive/Deep_learning/Eyelid_segmentation/MOBIUS/Masks"
    images_dir = "/content/drive/MyDrive/Deep_learning/Eyelid_segmentation/MOBIUS/Images"
    output_labels_dir = "/content/drive/MyDrive/Deep_learning/Eyelid_segmentation/MOBIUS/YOLOv8_training_LR/labels"
    output_images_dir = "/content/drive/MyDrive/Deep_learning/Eyelid_segmentation/MOBIUS/YOLOv8_training_LR/images"

    os.makedirs(output_labels_dir, exist_ok=True)
    os.makedirs(output_images_dir, exist_ok=True)

    num_points_eyelid = 24
    num_points_iris = 16
    num_points_pupil = 12

    start_folder = 0

    process_all_images_in_directory(masks_dir, images_dir, output_labels_dir, output_images_dir, num_points_eyelid, num_points_iris, num_points_pupil, start_folder)

Processing folder 35: 100%|██████████| 98/98 [03:35<00:00,  2.20s/it]


### 形式を調整

In [ ]:
### 枚数の確認

import os

def count_files(directory, include_bad=False):
    total_count = 0
    bad_count = 0
    for name in os.listdir(directory):
        if os.path.isfile(os.path.join(directory, name)):
            total_count += 1
            if 'bad' in name.lower():
                bad_count += 1
    return total_count, bad_count

# ディレクトリパス
labels_dir = "/content/drive/MyDrive/Deep_learning/Eyelid_segmentation/MOBIUS/YOLOv8_training_LR/labels/"
images_dir = "/content/drive/MyDrive/Deep_learning/Eyelid_segmentation/MOBIUS/YOLOv8_training_LR/images/"

# ファイル数をカウント
labels_total, labels_bad = count_files(labels_dir)
images_total, images_bad = count_files(images_dir)

print(f"ラベルファイル総数: {labels_total}")
print(f"'bad'を含むラベルファイル数: {labels_bad}")
print(f"画像ファイル総数: {images_total}")
print(f"'bad'を含む画像ファイル数: {images_bad}")

if labels_total == images_total:
    print("ラベルと画像の総数が一致しています。")
else:
    print("警告: ラベルと画像の総数が一致していません。")

if labels_bad == images_bad:
    print("'bad'を含むラベルと画像の数が一致しています。")
else:
    print("警告: 'bad'を含むラベルと画像の数が一致していません。")

In [ ]:
"""
このスクリプトは以下の処理を行います：

1. 指定されたソースディレクトリ（画像とラベル）からファイルを読み込みます。
2. ファイル名に'bad'が含まれていないファイルをフィルタリングします。
3. フィルタリングされたファイルを訓練用と検証用に分割します（デフォルトで80:20の比率）。
4. 以下の新しい構造でディレクトリを作成し、ファイルをコピーします：
   /content/drive/MyDrive/Deep_learning/Eyelid_segmentation/MOBIUS/YOLOv8_training/data/
   ├── images/
   │   ├── train/
   │   └── val/
   └── labels/
       ├── train/
       └── val/
5. tqdmを使用して、ファイルコピーの進行状況を表示します。
6. 処理されたファイル数を報告し、画像とラベルの数が一致しているか確認します。

注意：
- このスクリプトは既存のファイルを上書きします。実行前に出力ディレクトリの内容を確認してください。
- 大量のファイルを扱う場合でも、tqdmにより進行状況を確認できます。
"""

import os
import shutil
from sklearn.model_selection import train_test_split
from tqdm import tqdm

def copy_files(src_dir, dest_dir, files):
    os.makedirs(dest_dir, exist_ok=True)
    for file in tqdm(files, desc=f"Copying to {os.path.basename(dest_dir)}", unit="file"):
        shutil.copy2(os.path.join(src_dir, file), dest_dir)

def process_directory(src_dir, train_dir, val_dir, val_split=0.2):
    print(f"処理中のディレクトリ: {src_dir}")
    files = [f for f in os.listdir(src_dir) if os.path.isfile(os.path.join(src_dir, f)) and 'bad' not in f.lower()]

    train_files, val_files = train_test_split(files, test_size=val_split, random_state=42)

    copy_files(src_dir, train_dir, train_files)
    copy_files(src_dir, val_dir, val_files)

    return len(train_files), len(val_files)

# ベースディレクトリ
base_dir = "/content/drive/MyDrive/Deep_learning/Eyelid_segmentation/MOBIUS/YOLOv8_training_LR"

# ソースディレクトリ
src_images_dir = os.path.join(base_dir, "images")
src_labels_dir = os.path.join(base_dir, "labels")

# 新しい出力ディレクトリ構造
output_dir = os.path.join(base_dir, "data")
images_dir = os.path.join(output_dir, "images")
labels_dir = os.path.join(output_dir, "labels")

# 新しいディレクトリ構造の作成
train_images_dir = os.path.join(images_dir, "train")
val_images_dir = os.path.join(images_dir, "val")
train_labels_dir = os.path.join(labels_dir, "train")
val_labels_dir = os.path.join(labels_dir, "val")

# ファイルの処理とコピー
print("処理を開始します...")
train_images, val_images = process_directory(src_images_dir, train_images_dir, val_images_dir)
train_labels, val_labels = process_directory(src_labels_dir, train_labels_dir, val_labels_dir)

print(f"\n処理完了:")
print(f"訓練用画像: {train_images}")
print(f"検証用画像: {val_images}")
print(f"訓練用ラベル: {train_labels}")
print(f"検証用ラベル: {val_labels}")

if train_images == train_labels and val_images == val_labels:
    print("画像とラベルの数が一致しています。")
else:
    print("警告: 画像とラベルの数が一致していません。")

print(f"\n新しいデータセットの場所: {output_dir}")
print("新しいディレクトリ構造:")
print(f"{output_dir}/")
print("├── images/")
print("│   ├── train/")
print("│   └── val/")
print("└── labels/")
print("    ├── train/")
print("    └── val/")

In [ ]:
"""
このスクリプトは以下の処理を行います：

1. 指定されたベースディレクトリ内の 'images' と 'labels' フォルダを削除します。
2. 削除前にユーザーに確認を求めます。
3. 削除処理の結果を報告します。

注意：
- このスクリプトは元のデータを完全に削除します。実行前に十分注意し、必要なデータのバックアップを確保してください。
- 削除操作は取り消せません。確認プロンプトには慎重に回答してください。
"""

import os
import shutil

def remove_directory(dir_path):
    try:
        shutil.rmtree(dir_path)
        print(f"削除完了: {dir_path}")
    except Exception as e:
        print(f"エラー: {dir_path}の削除中に問題が発生しました。\n{str(e)}")

# ベースディレクトリ
base_dir = "/content/drive/MyDrive/Deep_learning/Eyelid_segmentation/MOBIUS/YOLOv8_training_LR"

# ソースディレクトリ
src_images_dir = os.path.join(base_dir, "images")
src_labels_dir = os.path.join(base_dir, "labels")

print(f"削除対象のディレクトリ:")
print(f"1. {src_images_dir}")
print(f"2. {src_labels_dir}")

# 削除の確認
confirm = input("\n上記のフォルダを削除しますか？ この操作は取り消せません。(y/n): ")

if confirm.lower() == 'y':
    remove_directory(src_images_dir)
    remove_directory(src_labels_dir)
    print("\n指定されたフォルダの削除が完了しました。")
else:
    print("\n削除操作がキャンセルされました。フォルダは削除されていません。")

In [ ]:
# YAMLファイルの作成

%%writefile /content/drive/MyDrive/Deep_learning/Eyelid_segmentation/MOBIUS/YOLOv8_training_LR/eye_segmentation.yaml
# Train/val/test sets as 1) dir: path/to/imgs, 2) file: path/to/imgs.txt, or 3) list: [path/to/imgs1, path/to/imgs2, ..]
path: /content/drive/MyDrive/Deep_learning/Eyelid_segmentation/MOBIUS/YOLOv8_training_LR/data  # dataset root dir
train: images/train  # train images (relative to 'path')
val: images/val  # val images (relative to 'path')
test: images/val # test images (optional)

# Classes
names:
  0: eyelid_R
  1: iris_R
  2: pupil_R
  3: eyelid_L
  4: iris_L
  5: pupil_L

# Download script/URL (optional)
download: https://ultralytics.com/assets/coco128-seg.zip

#**YOLOv8 training**

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
%cd /content/drive/MyDrive/Deep_learning/Eyelid_segmentation/MOBIUS/YOLOv8_training_LR
!git clone https://github.com/ultralytics/ultralytics
%cd ultralytics
#yolov8

In [ ]:
from ultralytics import YOLO
#本体
#nc = 3に変更
# Load a model
pretrained_path = "/content/drive/MyDrive/Deep_learning/Eyelid_segmentation/MOBIUS/YOLOv8_training/ultralytics/runs/segment/train3_458epochs/weights/best.pt"
#model = YOLO('yolov8n-seg.yaml').load('yolov8n-seg.pt')  # build from YAML and transfer weights
model = YOLO('yolov8n-seg.yaml').load('pretrained_path')  # build from YAML and transfer weights

In [ ]:
result =  model.train(data='/content/drive/MyDrive/Deep_learning/Eyelid_segmentation/MOBIUS/YOLOv8_training/eye_segmentation.yaml', epochs=30, imgsz=640)

In [ ]:
## 続きからトレーニングする場合
from ultralytics import YOLO
#本体
#nc = 3に変更
# Load a model
model = YOLO('yolov8n-seg.yaml').load('yolov8n-seg.pt')  # build from YAML and transfer weights

result = model.train(data='/content/drive/MyDrive/Deep_learning/Eyelid_segmentation/MOBIUS/YOLOv8_training/eye_segmentation.yaml',
                     epochs=500,
                     imgsz=640,
                     resume='/content/drive/MyDrive/Deep_learning/Eyelid_segmentation/MOBIUS/YOLOv8_training_LR/ultralytics/runs/segment/train/weights/last.pt')  # チェックポイントファイルのパスを指定